In [1]:
import numpy as np
import pandas as pd
import scipy.constants as consts
import matplotlib.pyplot as plt
from io import StringIO
import glob
import os, sys
import re
plt.rcParams['figure.dpi']= 300

In [2]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 5),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)
# set wanted parameters for plots

# e) Interaction case

Read in energies:

In [27]:
def name_dat_file(directory):
    filenames = [f for f in glob.glob(directory + '*.dat') if 'Eint' in f]
    return filenames

In [37]:
def name_dat_filemain(directory):
    filenames = [f for f in glob.glob(directory + '*.dat') if not 'Eint' in f]
    return filenames

Sort the filenames nicely:

In [4]:
def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]"""
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    #Sort the given list in the way that humans expect
    l.sort(key=alphanum_key)
    return l

## Read main short file in to a dataframe:

In [5]:
def matchlist_iter(filename):
    column_names = ['energy', 'acceptance', 'timecpu','solver']
    df = pd.read_csv(filename,skiprows=5,names=column_names,delim_whitespace=True,comment='#')
    return df

## Read in energy files:

In [81]:
def read_Ean_2_df(filenamesmain,fileE,Ns,dims=3):
    #creates a new dataframe that's empty
    mainDF = pd.DataFrame()
    #column_names = [['energy']]
    print(Ns)
    energy = pd.DataFrame()
    for i in range(len(filenamesmain)):
        temp = pd.DataFrame()
        main = matchlist_iter(filenamesmain[i])
        main = main.assign(N=Ns[i]*np.ones(len(main)))
        main = main.assign(dim=dims * np.ones(len(main)))
        temp['energy'] = pd.read_csv(fileE[i])
        temp = temp.assign(N=Ns[i]*np.ones(len(temp)))
        temp = temp.assign(dim=dims * np.ones(len(temp)))
        mainDF = mainDF.append(main,ignore_index=True)
        energy = energy.append(temp,ignore_index=True)
    return mainDF, energy


In [90]:
def read_E_alpha(filenamesmain,fileE,Ns):
    #creates a new dataframe that's empty
    mainDF = pd.DataFrame()
    alphas = np.linspace(0.3,0.7,5)
    print(Ns)
    energy = pd.DataFrame()
    for i in range(len(alphas)):
        temp = pd.DataFrame()
        main = matchlist_iter(filenamesmain[i])
        main = main.assign(N=Ns*np.ones(len(main)))
        main = main.assign(alpha=alphas[i] * np.ones(len(main)))
        temp['energy'] = pd.read_csv(fileE[i])
        temp = temp.assign(N=Ns*np.ones(len(temp)))
        temp = temp.assign(alpha=alphas[i] * np.ones(len(temp)))
        mainDF = mainDF.append(main,ignore_index=True)
        energy = energy.append(temp,ignore_index=True)
    return mainDF, energy

### Make the dataframe drop uneccesary columns

In [7]:
def col_on_u(data):
    newcols = ['alpha', 'E_mean_an', 'std_b','acceptance', 'timecpu']
    return data[newcols]
 

In [70]:
def col_on_me(data):
    newcols = ['N','energy', 'std_b','acceptance', 'timecpu']
    return data[newcols]

In [169]:
def col_on_dude(data):
    newcols = ['alpha','N','benchmark','energy', 'std_b','acceptance', 'timecpu']
    return data[newcols]

## Blocking

In [8]:
def block_mean(vec):
    return sum(vec)/len(vec)

def meanAndVariance(vec):
    mean = np.mean(vec)
    var = sum([i ** 2 for i in vec])/len(vec) - mean*mean
    return mean, var

In [9]:
def everything_is_awesome(data): # does the blocking on the dataframe
    n_blocks = 200
    block_size_min = 100
    block_size_max = len(data)/100
    block_step = int ((block_size_max - block_size_min + 1) / n_blocks)
    mean_vec = []
    var_vec = []
    block_sizes = []
    for i in range(0, n_blocks):
        mean_temp_vec = []
        start_point = 0
        end_point = block_size_min + block_step*i
        block_size = end_point
        block_sizes.append(block_size)

    mean_temp_vec.append(block_mean(data[start_point:end_point]))
    start_point = end_point
    end_point += block_size_min + block_step*i
    mean, var = meanAndVariance(mean_temp_vec)
    mean_vec.append(mean)
    var_vec.append(np.sqrt(var/(len(data)/float(block_size) - 1.0)))

    mean, var = meanAndVariance(data)

    return mean,var

#### Add $\sigma_b$ to a dataframe

In [66]:
def add_block(dfmain,dfen,n):
    tempmean = 0
    tempvar = 0
    meanlist = []
    stdlist = []
    for i in range(len(n)):
        tempmean, tempvar = everything_is_awesome(dfen[dfen.N==n[i]]['energy'])
        meanlist.append(tempmean)
        stdlist.append(np.sqrt(np.abs((tempvar))))
    dfmain['std_b'] = stdlist
    return dfmain

In [149]:
def add_block_alpha(dfmain,dfen):
    tempmean = 0
    tempvar = 0
    meanlist = []
    stdlist = []
    alphas=np.linspace(0.3,0.7,5)
    for i in range(len(alphas)):
        tempmean, tempvar = everything_is_awesome(dfen[dfen.alpha==alphas[i]]['energy'])
        meanlist.append(tempmean)
        stdlist.append(np.sqrt(np.abs((tempvar))))
    dfmain['std_b'] = stdlist
    return dfmain

## Make $\LaTeX$ tables

In [141]:
def latex(df,colname):
    df = df.rename(columns=dict(zip(df, colname)))
    table = df.to_latex(index=False,escape=False,column_format=(1+ len(colname))*'c')
    table = table.replace("toprule", "hline \hline")
    table = table.replace("bottomrule", "hline \hline")
    table = table.replace("midrule", "hline")
    s = r'''\begin{table}[H]
    \centering
    \caption{}
    \label{tab:}
    '''
    table = s + table.replace("     "," ")
    table = table +'\end{table}'
    return table

In [215]:
def latex2(df,colname):
    df = df.rename(columns=dict(zip(df, colname)))
    table = df.to_latex(index=True,escape=False,column_format=(1+ len(colname))*'c')
    table = table.replace("toprule", "hline \hline")
    table = table.replace("bottomrule", "hline \hline")
    table = table.replace("midrule", "hline")
    s = r'''\begin{table}[H]
    \centering
    \caption{}
    \label{tab:}
    '''
    table = s + table.replace("     "," ")
    table = table +'\end{table}'
    return table

filepath for e)

In [20]:
filepath = ['e/e_a0/','e/e_n10_alphas/','e/e_n50_alphas/','e/e_n100_alphas/']

find all .dat files

In [225]:
filesE = []
filesm = []
for i in range(len(filepath)):
    filesE.append(name_dat_file(filepath[i]))
    filesm.append(name_dat_filemain(filepath[i]))
    sort_nicely(filesE[i])
    sort_nicely(filesm[i])

In [226]:
#sort_nicely(filesE)
filesE

[['e/e_a0/Eint_a0_e_a0_n10_N10.dat',
  'e/e_a0/Eint_a0_e_a0_n50_N50.dat',
  'e/e_a0/Eint_e_a0_n100_alpha0.500000.dat'],
 ['e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.300000.dat',
  'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.400000.dat',
  'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.500000.dat',
  'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.600000.dat',
  'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.700000.dat'],
 ['e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.300000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.400000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.500000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.600000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.700000.dat'],
 ['e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.300000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.400000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.500000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.600000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.700000.dat']]

In [44]:
filesm

[['e/e_a0/e_a0_n10.dat',
  'e/e_a0/e_a0_n50.dat',
  'e/e_a0/e_a0_n100_alpha0.500000.dat'],
 ['e/e_n10_alphas/e_n10_alpha0.300000.dat',
  'e/e_n10_alphas/e_n10_alpha0.400000.dat',
  'e/e_n10_alphas/e_n10_alpha0.500000.dat',
  'e/e_n10_alphas/e_n10_alpha0.600000.dat',
  'e/e_n10_alphas/e_n10_alpha0.700000.dat'],
 ['e/e_n50_alphas/e_n50_alpha0.300000.dat',
  'e/e_n50_alphas/e_n50_alpha0.400000.dat',
  'e/e_n50_alphas/e_n50_alpha0.500000.dat',
  'e/e_n50_alphas/e_n50_alpha0.600000.dat',
  'e/e_n50_alphas/e_n50_alpha0.700000.dat'],
 ['e/e_n100_alphas/e_n100.dat',
  'e/e_n100_alphas/e_n100_alpha0.300000.dat',
  'e/e_n100_alphas/e_n100_alpha0.400000.dat',
  'e/e_n100_alphas/e_n100_alpha0.600000.dat',
  'e/e_n100_alphas/e_n100_alpha0.700000.dat']]

In [50]:
N = [10,50,100] # particles we're working on
alpha = [0.3,0.4,0.5,0.6,0.7] # alpha's we're looking at

sort a = 0 into a dataframe

In [46]:
a0listE = filesE[0]
a0listm = filesm[0]


['e/e_a0/Eint_a0_e_a0_n10_N10.dat',
 'e/e_a0/Eint_a0_e_a0_n50_N50.dat',
 'e/e_a0/Eint_e_a0_n100_alpha0.500000.dat']

In [59]:
a0m, a0E = read_Ean_2_df(a0listm,a0listE,N)

[10, 50, 100]


In [67]:
a0m = add_block(dfen=a0E,dfmain=a0m,n=N)

In [68]:
a0m

,energy,acceptance,timecpu,solver,N,dim,std_b
0,24.0849,0.892023,27.96046,3,10.0,3.0,0.000135
1,120.4979,0.228547,534.51240,3,50.0,3.0,0.000545
2,241.0094,0.032484,2066.11200,3,100.0,3.0,0.000887


In [71]:
a0m = col_on_me(a0m)

In [72]:
a0m

,N,energy,std_b,acceptance,timecpu
0,10.0,24.0849,0.000135,0.892023,27.96046
1,50.0,120.4979,0.000545,0.228547,534.51240
2,100.0,241.0094,0.000887,0.032484,2066.11200


Make $a = 0$ into a Latex table

In [73]:
col_name =[f"${v}$" for v in [r"N",r"\langle E_L \rangle", r"\sigma_b", r"\text{acceptance } [\%]", r"t_{CPU} [s]"]]

In [142]:
a0lat = latex(a0m,col_name)

In [143]:
print(a0lat)

\begin{table}[H]
    \centering
    \caption{}
    \label{tab:}
    \begin{tabular}{cccccc}
\hline \hline
   $N$ &  $\langle E_L \rangle$ &  $\sigma_b$ &  $\text{acceptance } [\%]$ &  $t_{CPU} [s]$ \\
\hline
  10.0 &    24.0849 &    0.000135 &       0.892023 &   27.96046 \\
  50.0 &   120.4979 &    0.000545 &       0.228547 &  534.51240 \\
 100.0 &   241.0094 &    0.000887 &       0.032484 & 2066.11200 \\
\hline \hline
\end{tabular}
\end{table}


In [144]:
#help(a0m.to_csv)

### Read in all the other energies when $a = 0.0043$

In [227]:
N10E = filesE[1]
N10mf = filesm[1]

In [228]:
N50E = filesE[2]
N50mf = filesm[2]

In [229]:
N100E = filesE[3]
N100mf = filesm[3]

In [230]:
N10E

['e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.300000.dat',
 'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.400000.dat',
 'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.500000.dat',
 'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.600000.dat',
 'e/e_n10_alphas/Eint_a_e_n10_N10_alpha0.700000.dat']

In [174]:
N10mf

['e/e_n10_alphas/e_n10_alpha0.300000.dat',
 'e/e_n10_alphas/e_n10_alpha0.400000.dat',
 'e/e_n10_alphas/e_n10_alpha0.500000.dat',
 'e/e_n10_alphas/e_n10_alpha0.600000.dat',
 'e/e_n10_alphas/e_n10_alpha0.700000.dat']

In [231]:
N100mf

['e/e_n100_alphas/e_n100_alpha0.300000.dat',
 'e/e_n100_alphas/e_n100_alpha0.400000.dat',
 'e/e_n100_alphas/e_n100_alpha0.500000.dat',
 'e/e_n100_alphas/e_n100_alpha0.600000.dat',
 'e/e_n100_alphas/e_n100_alpha0.700000.dat']

In [175]:
N10m, E10 = read_E_alpha(N10mf,N10E,10)

10


In [176]:
N10m

,energy,acceptance,timecpu,solver,N,alpha
0,14.99666,0.955622,25.94881,3,10.0,0.3
1,19.55271,0.944180,27.25675,3,10.0,0.4
2,24.22702,0.900235,27.73884,3,10.0,0.5
3,28.85555,0.933428,28.33244,3,10.0,0.6
4,33.56793,0.804983,31.39450,3,10.0,0.7


In [177]:
N50m, E50 = read_E_alpha(N50mf,N50E,50)

50


In [178]:
N50m

,energy,acceptance,timecpu,solver,N,alpha
0,74.38858,0.867151,510.3081,3,50.0,0.3
1,97.66119,0.794913,539.4805,3,50.0,0.4
2,120.68420,0.607094,564.0068,3,50.0,0.5
3,144.59280,0.472968,576.4249,3,50.0,0.6
4,167.91910,0.450908,573.5652,3,50.0,0.7


In [232]:
N100m, E100 = read_E_alpha(N100mf,N100E,100)

100


In [233]:
N100m

,energy,acceptance,timecpu,solver,N,alpha
0,148.3572,0.787158,2115.342,3,100.0,0.3
1,194.9959,0.653655,2093.169,3,100.0,0.4
2,241.0076,0.534920,2114.789,3,100.0,0.5
3,288.1940,0.523795,2119.060,3,100.0,0.6
4,336.4776,0.056077,1951.804,3,100.0,0.7


Add blocking to the energies

In [181]:
N10m = add_block_alpha(N10m,E10)

In [182]:
N10m

,energy,acceptance,timecpu,solver,N,alpha,std_b
0,14.99666,0.955622,25.94881,3,10.0,0.3,0.227584
1,19.55271,0.944180,27.25675,3,10.0,0.4,0.182613
2,24.22702,0.900235,27.73884,3,10.0,0.5,0.353728
3,28.85555,0.933428,28.33244,3,10.0,0.6,0.282949
4,33.56793,0.804983,31.39450,3,10.0,0.7,1.625600


In [183]:
N50m = add_block_alpha(N50m,E50)

In [184]:
N50m

,energy,acceptance,timecpu,solver,N,alpha,std_b
0,74.38858,0.867151,510.3081,3,50.0,0.3,3.173188
1,97.66119,0.794913,539.4805,3,50.0,0.4,1.506749
2,120.68420,0.607094,564.0068,3,50.0,0.5,0.804185
3,144.59280,0.472968,576.4249,3,50.0,0.6,1.304669
4,167.91910,0.450908,573.5652,3,50.0,0.7,1.082076


In [234]:
N100m = add_block_alpha(N100m,E100)

In [235]:
N100m

,energy,acceptance,timecpu,solver,N,alpha,std_b
0,148.3572,0.787158,2115.342,3,100.0,0.3,3.687806
1,194.9959,0.653655,2093.169,3,100.0,0.4,1.307078
2,241.0076,0.534920,2114.789,3,100.0,0.5,1.153398
3,288.1940,0.523795,2119.060,3,100.0,0.6,0.850927
4,336.4776,0.056077,1951.804,3,100.0,0.7,2.704718


In [191]:
N10m['benchmark'] = [24.7,24.2,24.2,24.6,25.5]

In [192]:
N50m['benchmark'] = [138,125,122,125,129]

In [236]:
N100m['benchmark'] = [278,253,247,252,263]

In [237]:
N10m = col_on_dude(N10m)
N50m = col_on_dude(N50m)
N100m = col_on_dude(N100m)

In [198]:
N50m

,alpha,N,benchmark,energy,std_b,acceptance,timecpu
0,0.3,50.0,138,74.38858,3.173188,0.867151,510.3081
1,0.4,50.0,125,97.66119,1.506749,0.794913,539.4805
2,0.5,50.0,122,120.68420,0.804185,0.607094,564.0068
3,0.6,50.0,125,144.59280,1.304669,0.472968,576.4249
4,0.7,50.0,129,167.91910,1.082076,0.450908,573.5652


In [ ]:



allNsarebeautiful = pd.concat()

In [238]:
n10 = N10m.drop(['N'],axis=1)

In [239]:
n50 = N50m.drop(['N'],axis=1)
n50

,alpha,benchmark,energy,std_b,acceptance,timecpu
0,0.3,138,74.38858,3.173188,0.867151,510.3081
1,0.4,125,97.66119,1.506749,0.794913,539.4805
2,0.5,122,120.68420,0.804185,0.607094,564.0068
3,0.6,125,144.59280,1.304669,0.472968,576.4249
4,0.7,129,167.91910,1.082076,0.450908,573.5652


In [240]:
n100 = N100m.drop(['N'],axis=1)

In [241]:
n100

,alpha,benchmark,energy,std_b,acceptance,timecpu
0,0.3,278,148.3572,3.687806,0.787158,2115.342
1,0.4,253,194.9959,1.307078,0.653655,2093.169
2,0.5,247,241.0076,1.153398,0.534920,2114.789
3,0.6,252,288.1940,0.850927,0.523795,2119.060
4,0.7,263,336.4776,2.704718,0.056077,1951.804


In [242]:
allN = pd.concat([n10,n50,n100],keys=['N=10','N=50','N=100'])

In [243]:
Nl = allN.to_latex(index=True)

In [244]:
print(Nl)

\begin{tabular}{llrrrrrr}
\toprule
     &   &  alpha &  benchmark &     energy &     std\_b &  acceptance &     timecpu \\
\midrule
N=10 & 0 &    0.3 &       24.7 &   14.99666 &  0.227584 &    0.955622 &    25.94881 \\
     & 1 &    0.4 &       24.2 &   19.55271 &  0.182613 &    0.944180 &    27.25675 \\
     & 2 &    0.5 &       24.2 &   24.22702 &  0.353728 &    0.900235 &    27.73884 \\
     & 3 &    0.6 &       24.6 &   28.85555 &  0.282949 &    0.933428 &    28.33244 \\
     & 4 &    0.7 &       25.5 &   33.56793 &  1.625600 &    0.804983 &    31.39450 \\
N=50 & 0 &    0.3 &      138.0 &   74.38858 &  3.173188 &    0.867151 &   510.30810 \\
     & 1 &    0.4 &      125.0 &   97.66119 &  1.506749 &    0.794913 &   539.48050 \\
     & 2 &    0.5 &      122.0 &  120.68420 &  0.804185 &    0.607094 &   564.00680 \\
     & 3 &    0.6 &      125.0 &  144.59280 &  1.304669 &    0.472968 &   576.42490 \\
     & 4 &    0.7 &      129.0 &  167.91910 &  1.082076 &    0.450908 &   573.56520 \

In [245]:
colnames =[f"${v}$" for v in [r"\alpha",r"\text{benchmark}",r"\langle E_L \rangle", r"\sigma_b", r"\text{acceptance } [\%]", r"t_{CPU} [s]"]]
Nl2 = latex2(allN,colnames)

In [246]:
print(Nl2)

\begin{table}[H]
    \centering
    \caption{}
    \label{tab:}
    \begin{tabular}{ccccccc}
\hline \hline
 &   &  $\alpha$ &  $\text{benchmark}$ &  $\langle E_L \rangle$ &  $\sigma_b$ &  $\text{acceptance } [\%]$ &  $t_{CPU} [s]$ \\
\hline
N=10 & 0 &   0.3 &    24.7 &   14.99666 &    0.227584 &       0.955622 &   25.94881 \\
 & 1 &   0.4 &    24.2 &   19.55271 &    0.182613 &       0.944180 &   27.25675 \\
 & 2 &   0.5 &    24.2 &   24.22702 &    0.353728 &       0.900235 &   27.73884 \\
 & 3 &   0.6 &    24.6 &   28.85555 &    0.282949 &       0.933428 &   28.33244 \\
 & 4 &   0.7 &    25.5 &   33.56793 &    1.625600 &       0.804983 &   31.39450 \\
N=50 & 0 &   0.3 &   138.0 &   74.38858 &    3.173188 &       0.867151 &  510.30810 \\
 & 1 &   0.4 &   125.0 &   97.66119 &    1.506749 &       0.794913 &  539.48050 \\
 & 2 &   0.5 &   122.0 &      120.68420 &    0.804185 &       0.607094 &  564.00680 \\
 & 3 &   0.6 &   125.0 &      144.59280 &    1.304669 &       0.472968 &  576.42490

####  Make a nice table for when $a = 0$ and $\beta = 1$